# Testing individual functions

## Test: filtering duplicate boxes

In [ ]:
# Test 'filter_detections_xyxy'
from scg_detection_tools.models import from_type
from scg_detection_tools.detect import Detector
from scg_detection_tools.filters import DetectionFilterDuplicates
import scg_detection_tools.utils.image_tools as imtools
import matplotlib.pyplot as plt
import cv2

model = from_type("yolov8", "/home/juliocesar/leaf-detection/checkpoints/yolov10/large/train6/best.pt")
# model = from_type("yolonas", "/home/juliocesar/leaf-detection/checkpoints/yolonas/large/train8/ckpt_best.pth")
detector = Detector(model)
detector.update_parameters(confidence=50.0, slice_detect=False, slice_wh=(480,480), slice_fill=False)

imgs = [
    "/home/juliocesar/leaf-detection/imgs/analysis_test/0_640x640.png",
    "/home/juliocesar/leaf-detection/imgs/analysis_test/1_640x640.png",
    "/home/juliocesar/leaf-detection/imgs/analysis_test/2_640x640_aglom.png",
]
detections = detector(imgs)

_, axs = plt.subplots(ncols=len(imgs), figsize=(24,15))
for i, (img, det) in enumerate(zip(imgs, detections)):
    axs[i].axis("off")
    orig_img = cv2.imread(img)
    orig_img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
    
    axs[i].imshow(imtools.box_annotated_image(orig_img, det))
    axs[i].set_title("Before filtering box")
plt.show()

for i in range(len(detections)):
    filter = DetectionFilterDuplicates(intersection_tresh=0.8, imghw=cv2.imread(imgs[i]).shape[:2])
    detections[i] = filter(detections[i])

_, axs = plt.subplots(ncols=len(imgs), figsize=(24,15))
for i, (img, det) in enumerate(zip(imgs, detections)):
    axs[i].axis("off")
    orig_img = cv2.imread(img)
    orig_img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
    
    axs[i].imshow(imtools.box_annotated_image(orig_img, det))
    axs[i].set_title("After filtering box")
plt.show()


## Test new dataset generation code

In [ ]:
%set_env CUDA_VISIBLE_DEVICES=1

In [ ]:
from scg_detection_tools.generate import DatasetGenerator
from scg_detection_tools.models import from_type
from scg_detection_tools.utils.file_handling import get_all_files_from_paths

det_params = {
    "confidence": 50.0,
    "slice_detect": True,
    "slice_wh": (640,640),
    "slice_fill": True,
    "detection_filters": { "duplicate_filter": True, "duplicate_filter_thresh": 0.95, "object_size_filter": True, "object_size_max_wh": (80,80), }
}
MODEL_PATH = "/home/juliocesar/leaf-detection/checkpoints/yolov10/large/train6/best.pt"
MODEL_TYPE = "yolov8"
# MODEL_PATH = "/home/juliocesar/leaf-detection/checkpoints/yolonas/large/train7/ckpt_best.pth"
# MODEL_TYPE = "yolonas"
model = from_type(MODEL_TYPE, MODEL_PATH)

# IMG_DIR = "/home/juliocesar/leaf-detection/imgs/newtodataset"
# imgs = get_all_files_from_paths(IMG_DIR)
imgs = ["/home/juliocesar/leaf-detection/imgs/rawimgs/l.png"]

generator = DatasetGenerator(
    img_files=imgs,
    class_labels=["leaf"],
    model=model,
    annotation_type="box",
    sam2_path="/home/juliocesar/leaf-detection/checkpoints/sam2/sam2_hiera_tiny.pt",
    detection_parameters=det_params,
    save_on_slice=True,
    # on_slice_resize=(640,640),
    augmentation_steps=None,
)
generator.generate(save_on_finish=True)